In [6]:
from flu_model import FluModel, State
import numpy as np
import pandas as pd
from ipywidgets.embed import embed_minimal_html

# ipywidgets for UI
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Label

In [7]:
def flu_simulation(pop, x_map, y_map, s, opts, N_steps, biased, init_inf, plots=[True, True, True, True, True]):
    
    # generate model 
    model = FluModel(pop, width=x_map, height=y_map, init_inf=init_inf, biased=biased)
    
    # initialize color map and empty lists
    factors = ['State.SUSCEPTIBLE', 'State.INFECTED', 'State.RECOVERED']
    deceased = []; sampled = []
    
    for i in range(N_steps):
        
        # advance the flu model
        model.step()

        # append number of deceased from model to list
        deceased.append(model.deceased)

        # sample data for plotting 
        sampled.append(model.sample(s,i) * 100/s)
        
    # OUTPUT .CSV OF RESULTS
    agent_state = model.datacollector.get_agent_vars_dataframe().drop(columns='p')

    # change into counts of categorical data, rename columns
    df = pd.pivot_table(agent_state.reset_index(),index='Step',columns='State',aggfunc=np.size, fill_value=0)
    df.columns = ['Susceptible', 'Infected', 'Recovered']
    
    # add deceased and sampled (dashed-line) data to the dataframe
    df['Deceased'] = deceased
    df['Sampled'] = sampled
            
    df.to_csv('output.csv')
    print('output complete!')

In [8]:
## USER INTERFACE

# create buttons 
button = widgets.Button(description='Run')
button2 = widgets.Button(description='Fit to Population')

# gather selected values upon button push // submission
def launch_flu_simulation(b=None):
            
    # get checkbox values + assemble for plotting
    su_cb = susceptible.value
    in_cb = infected.value
    re_cb = recovered.value
    de_cb = deceased.value
    sa_cb = sampled.value
    
    plots = [su_cb, in_cb, re_cb, de_cb, sa_cb]        
        
    # build opts for plotting
    opts = dict(plot_width=490, plot_height=490, min_border=0, toolbar_location=None)
    
    # launch mesa simulation
    flu_simulation(pop=population_box.value, x_map=map_size_x.value, y_map=map_size_y.value,
                       s=sample_slider.value, opts=opts, N_steps=steps_box.value, 
                       init_inf=infected_slider.value, plots=plots, biased=biased_check.value)

# button click actions
@button.on_click
def plot_on_click(b):
    launch_flu_simulation()
    
@button2.on_click
def calc_on_click(b):
    
    # get population and find appropriate square dimensions
    pop = population_box.value
    dim = int(np.ceil(np.sqrt(pop) * 1.4))
    
    # update fields
    map_size_x.value = dim
    map_size_y.value = dim

# set up first tab input fields
sample_slider = widgets.FloatSlider(
    value=1,
    min=0,
    max=100.0,
    step=0.1,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

biased_check = widgets.Checkbox()

# first tab for sampling info
tab1 = VBox(children=[HBox([Label('Percent Sampled'), sample_slider]),
                      HBox([Label('Biased Sampling'), biased_check])])

# set up second tab input fields
population_box = widgets.BoundedIntText(
    value=300,
    min=10,
    max=10000,
    step=1,
    orientation='horizontal'
)

steps_box = widgets.BoundedIntText(
    value=90,
    min=10,
    max=300,
    step=1,
    orientation='horizontal'
)

infected_slider = widgets.FloatSlider(min=0.1, max=5, step=0.1, value=1)

map_size_x = widgets.BoundedIntText(
    value=25,
    max=1000,
    min=10,
    step=1,
    orientation='horizontal'
)

map_size_y = widgets.BoundedIntText(
    value=25,
    min=10,
    max=1000,
    step=1,
    orientation='horizontal'
)

# second tab for model details
tab2 = VBox(children=[HBox([Label('Population'), population_box]), HBox([Label('Number of Timesteps'), steps_box]),
                              HBox([Label('Initial Infected Percent'), infected_slider]),
                              HBox([Label('Map Size'), map_size_x, Label(' x '), map_size_y, button2])])

# set up third tab input fields
susceptible = widgets.Checkbox(value=True)
infected = widgets.Checkbox(value=True)
recovered = widgets.Checkbox(value=True)
deceased = widgets.Checkbox(value=True)
sampled = widgets.Checkbox(value=True)

output_fn = widgets.Text(
    placeholder='Enter filename',
    disabled=False
)

# third tab for plot displays
tab3 = HBox([VBox([Label('Susceptible'), Label('Infected'), Label('Recovered'), Label('Deceased'), Label('Sampled')]),
             VBox([susceptible, infected, recovered, deceased, sampled])])

# create complete user interface
tab = widgets.Tab(children=[tab1, tab2, tab3])
tab.set_title(0, 'Sampling')
tab.set_title(1, 'Model')
tab.set_title(2, 'Plotting')
VBox(children=[tab, button])

output complete!
